In [ ]:
# --- Provided by freeCodeCamp ---
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.5/614.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.9 MB/s eta 0:00:00
2.18.0-dev20240906


In [ ]:
# --- Provided by freeCodeCamp ---
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-09-07 12:45:36--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.1s    

2024-09-07 12:45:36 (3.12 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-09-07 12:45:36--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
# load data to pandas
col_names = ['Class', 'Message']
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=col_names)
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=col_names)


In [ ]:
df_train

,Class,Message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [ ]:
df_test

,Class,Message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [ ]:
df_train.loc[:, 'Message'].head()

,Message
0,ahhhh...just woken up!had a bad dream about u ...
1,you can never do nothing
2,"now u sound like manky scouse boy steve,like! ..."
3,mum say we wan to go then go... then she can s...
4,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
# make token
tokenizer_train = keras.preprocessing.text.Tokenizer()
tokenizer_train.fit_on_texts(df_train.loc[:, 'Message'])

# to sequences
seq_train = tokenizer_train.texts_to_sequences(df_train.loc[:, 'Message'])
seq_test = tokenizer_train.texts_to_sequences(df_test.loc[:, 'Message'])

# padding
max_len = 20
X_train = keras.preprocessing.sequence.pad_sequences(seq_train, maxlen=max_len)
X_test = keras.preprocessing.sequence.pad_sequences(seq_test, maxlen=max_len)

In [ ]:
df_train['Class'].unique()

array(['ham', 'spam'], dtype=object)

In [ ]:
df_test['Class'].unique()

array(['ham', 'spam'], dtype=object)

In [ ]:
# encode Class
Y_train = (df_train['Class'] == 'ham').astype(int)
Y_test = (df_test['Class'] == 'ham').astype(int)

In [ ]:
Y_train.head()

,Class
0,1
1,1
2,1
3,1
4,1


In [ ]:
Y_test.head()

,Class
0,1
1,1
2,1
3,1
4,1


In [ ]:
# create model
model = tf.keras.Sequential()
# add embedding layer
word_count = len(tokenizer_train.word_index) + 1
model.add(tf.keras.layers.Embedding(input_dim=word_count, output_dim=100, input_length=max_len))
# add cnn layer
model.add(tf.keras.layers.Conv1D(filters=8, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
# add dense layer
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
# conpile and traning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7535 - loss: 0.5663 - val_accuracy: 0.9737 - val_loss: 0.1585
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9844 - loss: 0.0962 - val_accuracy: 0.9892 - val_loss: 0.0562
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9948 - loss: 0.0310 - val_accuracy: 0.9880 - val_loss: 0.0411
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9979 - loss: 0.0104 - val_accuracy: 0.9892 - val_loss: 0.0414
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9993 - loss: 0.0034 - val_accuracy: 0.9880 - val_loss: 0.0434
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9994 - loss: 0.0053 - val_accuracy: 0.9904 - val_loss: 0.0408
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9998 - loss: 0.0022 - val_accuracy: 0.9904 - val_loss: 0.0397
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9997 - loss: 0.0024 - val_accur

In [ ]:
model.evaluate(X_test, Y_test, verbose=2)

44/44 - 0s - 2ms/step - accuracy: 0.9871 - loss: 0.0551


[0.05512440204620361, 0.9870689511299133]

In [ ]:
# --- Provided by freeCodeCamp ---
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # --- My solution start ---
  pred_text = pd.Series(pred_text)
  # token
  tokenizer_predict = keras.preprocessing.text.Tokenizer()
  tokenizer_predict.fit_on_texts(pred_text)
  seq_predict = tokenizer_train.texts_to_sequences(pred_text)
  X_pred = keras.preprocessing.sequence.pad_sequences(seq_predict, maxlen=max_len)
  prediction = model.predict(X_pred).flatten()
  if prediction[0] >= 0.7:
    prediction = [prediction[0],'ham']
  else:
    prediction = [prediction[0], 'spam']
  # --- My solution end ---
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[0.9998454, 'ham']


In [ ]:
# --- Provided by freeCodeCamp ---
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False
    print(prediction)
  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0.9998454, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[0.96505284, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0.99997556, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
[0.0052695055, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[1.3461225e-06, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[0.99995655, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[0.99987817, 'ham']
You haven't passed yet. Keep trying.
